![Strategy for Solving a Machine Learning Problem](http://radimrehurek.com/data_science_python/drop_shadows_background.png)

In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
#import xgboost as xgb

In [10]:
user = "Aritra"

In [ ]:
#####################################################################
# Reading the Cleaned Training & Testing Data                       #
#####################################################################

In [11]:
train_data = pd.read_csv("../data/cleaned_train_data.csv")
test_data = pd.read_csv("../data/cleaned_test_data.csv")
x_columns = test_data.columns[1:677]
y_column = "SalePrice"
x_train_values, y_train_values = train_data[x_columns].values, train_data[y_column].values
xgb_train_values = train_data[np.append(y_column, x_columns)].values
x_test_values = test_data[x_columns].values

In [12]:
def generate_submission_file(predictions, Id, filename):
    results = pd.DataFrame()
    results["Id"] = Id
    results["SalePrice"] = predictions
    results.to_csv(filename, index=False)

# Classification and Regression #

There are two major types of supervised machine learning problems, called classification and regression. In classification, the goal is to predict a class label, which is a choice from a predefined
list of possibilities. Classification is sometimes separated into binary classification, which is the special case of distinguishing between exactly two classes, and multiclass classification, which is classification between more than two classes. You can think of binary classification as trying to answer a yes/no question. Classifying emails as either spam or not spam is an example of a binary classification problem. In this binary classification task, the yes/no question being asked would be “Is this email spam?”

For regression tasks, the goal is to predict a continuous number, or a floating-point number in programming terms (or real number in mathematical terms). Predicting a person’s annual income from their education, their age, and where they live is an example of a regression task. When predicting income, the predicted value is an
amount, and can be any number in a given range. Another example of a regression task is predicting the yield of a corn farm given attributes such as previous yields, weather, and number of employees working on the farm. The yield again can be an arbitrary number.

An easy way to distinguish between classification and regression tasks is to ask whether there is some kind of continuity in the output. If there is continuity between possible outcomes, then the problem is a regression problem. Think about predicting annual income. There is a clear continuity in the output. Whether a person makes \$40000 or \$40001 a year does not make a tangible difference, even though these are different amounts of money; if our algorithm predicts \$39999 or \$40001 when it should have predicted \$40000, we don’t mind that much. 

By contrast, for the task of recognizing the language of a website (which is a classification problem), there is no matter of degree. A website is in one language, or it is in another. There is no continuity between languages, and there is no language that is
between English and French.

# Generalization, Overfitting, and Underfitting #

In supervised learning, we want to build a model on the training data and then be able to make accurate predictions on new, unseen data that has the same characteristics as the training set that we used. If a model is able to make accurate predictions on unseen data, we say it is able to generalize from the training set to the test set. We want to build a model that is able to generalize as accurately as possible.

Usually we build a model in such a way that it can make accurate predictions on the training set. If the training and test sets have enough in common, we expect the model to also be accurate on the test set. However, there are some cases where this can go wrong. For example, if we allow ourselves to build very complex models, we can always be as accurate as we like on the training set.

The only measure of whether an algorithm will perform well on new data is the evaluation on the test set. However, intuitively 3 we expect simple models to generalize better to new data. Therefore, we always want to find the simplest model. Building a model that is too complex for the amount of information we have, is called overfitting. Overfitting occurs when you fit a model too closely to the particularities of the training set and obtain a model that works well on the training set but is not able to generalize to new data. On the other hand, if your model is too simple, you might not be able to capture all the aspects of and variability in the data, and your model will do badly even on the training set. Choosing too simple a model is called underfitting.

The more complex we allow our model to be, the better we will be able to predict on the training data. However, if our model becomes too complex, we start focusing too much on each individual data point in our training set, and the model will not generalize well to new data. There is a sweet spot in between that will yield the best generalization performance. This is the model we want to find.

# Supervised Machine Learning Algorithms #

We will now review the most popular machine learning algorithms and explain how they learn from data and how they make predictions. We will also discuss how the concept of model complexity plays out for each of these models, and provide an overview of how each algorithm builds a model. We will examine the strengths and weaknesses of each algorithm, and what kind of data they can best be applied to. We will
also explain the meaning of the most important parameters and options. 4 Many algorithms have a classification and a regression variant, and we will describe both.

It is not necessary to read through the descriptions of each algorithm in detail, but understanding the models will give you a better feeling for the different ways machine learning algorithms can work. This chapter can also be used as a reference guide, and you can come back to it when you are unsure about the workings of any of the algorithms.

In [13]:
#####################################################################
# (High Dimensional) Linear Regression                              #
#####################################################################

In [14]:
#####################################################################
## Scikit Learn                                                    ##
#####################################################################

In [15]:
from sklearn.linear_model import RidgeCV, LassoCV, LarsCV, LassoLarsCV, ElasticNetCV

## Linear Models ##

Linear models are a class of models that are widely used in practice and have been studied extensively in the last few decades, with roots going back over a hundred years. Linear models make a prediction using a linear function of the input features, which we will explain shortly.

### Ridge regression ###

Ridge regression is also a linear model for regression, so the formula it uses to make predictions is the same one used for ordinary least squares. In ridge regression, though, the coefficients (w) are chosen not only so that they predict well on the training data, but also to fit an additional constraint. We also want the magnitude of coefficients to be as small as possible; in other words, all entries of w should be close to zero. Intuitively, this means each feature should have as little effect on the outcome as possible (which translates to having a small slope), while still predicting well. This
constraint is an example of what is called regularization. Regularization means explicitly restricting a model to avoid overfitting. The particular kind used by ridge regression is known as L2 regularization.

A less complex model means worse performance on the training
set, but better generalization. As we are only interested in generalization performance, we should choose the Ridge model over the LinearRegression model.

In [18]:
ridge_model = RidgeCV(alphas=np.linspace(0.0001,0.001,10))
ridge_model.fit(x_train_values, y_train_values)
ridge_model_predictions = ridge_model.predict(x_test_values)
generate_submission_file(ridge_model_predictions, test_data["Id"], "../results/" + user + "_RidgeCV_2.csv")

The Ridge model makes a trade-off between the simplicity of the model (near-zero coefficients) and its performance on the training set. How much importance the model places on simplicity versus training set performance can be specified by the user, using the alpha parameter. In the previous example, we used the default parameter `alpha=1.0` . There is no reason why this will give us the best trade-off, though.
The optimum setting of alpha depends on the particular dataset we are using. Increasing `alpha` forces coefficients to move more toward zero, which decreases training set performance but might help generalization.

### Lasso ###

An alternative to Ridge for regularizing linear regression is Lasso . As with ridge regression, using the lasso also restricts coefficients to be close to zero, but in a slightly different way, called L1 regularization. The consequence of L1 regularization is that when using the lasso, some coefficients are exactly zero. This means some features are entirely ignored by the model. This can be seen as a form of automatic feature selection. Having some coefficients be exactly zero often makes a model easier to interpret, and can reveal the most important features of your model.

In [19]:
lasso_model = LassoCV()
lasso_model.fit(x_train_values, y_train_values)
lasso_model_predictions = lasso_model.predict(x_test_values)
generate_submission_file(lasso_model_predictions, test_data["Id"], "../results/" + user + "_LassoCV.csv")

In [ ]:
lassolars_model = LassoLarsCV()
lassolars_model.fit(x_train_values, y_train_values)
lassolars_model_predictions = lassolars_model.predict(x_test_values)
generate_submission_file(lassolars_model_predictions, test_data["Id"], "../results/" + user + "_LassoLarsCV.csv")

### Elastic Net ###

In practice, ridge regression is usually the first choice between these two models. However, if you have a large amount of features and expect only a few of them to be important, Lasso might be a better choice. Similarly, if you would like to have a model that is easy to interpret, Lasso will provide a model that is easier to understand, as it will select only a subset of the input features. scikit-learn also provides the ElasticNet class, which combines the penalties of Lasso and Ridge. In practice, this combination works best, though at the price of having two parameters to adjust: one for the L1 regularization, and one for the L2 regularization.

In [22]:
en_model = ElasticNetCV(l1_ratio=0.75)
en_model.fit(x_train_values, y_train_values)
en_model_predictions = en_model.predict(x_test_values)
generate_submission_file(en_model_predictions, test_data["Id"], "../results/" + user + "_ElasticNetCV_1.csv")

## k-Nearest Neighbors ##

The k-NN algorithm is arguably the simplest machine learning algorithm. Building the model consists only of storing the training dataset. To make a prediction for a new data point, the algorithm finds the closest data points in the training dataset—its “nearest neighbors.”

**Strengths, weaknesses, and parameters:** In principle, there are two important parameters to the KNeighbors classifier: the number of neighbors and how you measure distance between data points. In practice, using a small number of neighbors like three or five often works well, but you should certainly adjust this parameter. Choosing the right distance measure is somewhat beyond the scope of this presentation. By default, Euclidean distance is used, which works
well in many settings. 

One of the strengths of k-NN is that the model is very easy to understand, and often gives reasonable performance without a lot of adjustments. Using this algorithm is a good baseline method to try before considering more advanced techniques. Building the nearest neighbors model is usually very fast, but when your training set is very large (either in number of features or in number of samples) prediction can be slow. When using the k-NN algorithm, it’s important to preprocess your data. This approach often does not perform well on datasets with many features (hundreds or more), and it does particularly badly with datasets where most features are 0 most of the time (so-called sparse datasets).

So, while the nearest k-neighbors algorithm is easy to understand, it is not often used in practice, due to prediction being slow and its inability to handle many features.

In [ ]:
#####################################################################
# Nearest Neighbors                                                 #
#####################################################################

In [ ]:
#####################################################################
## Scikit Learn                                                    ##
#####################################################################

In [ ]:
knn_model = KNeighborsRegressor()
knn_model.fit(x_train_values, y_train_values)
knn_model_predictions = knn_model.predict(x_test_values)
generate_submission_file(knn_model_predictions, test_data["Id"], "../results/" + user + "_KNN.csv")

In [ ]:
param_list = {"n_neighbors": [2, 4, 6]}
knn_gridsearch = GridSearchCV(KNeighborsRegressor(), param_list)
knn_gridsearch.fit(x_train_values, y_train_values)
knn_best_model_predictions = knn_gridsearch.best_estimator_.predict(x_test_values)
generate_submission_file(knn_best_model_predictions, test_data["Id"], "../results/" + user + "_KNN_GridSearchCV.csv")

## Decision Tree ##

In [ ]:
#####################################################################
# Decision Trees                                                    #
#####################################################################

In [ ]:
#####################################################################
## Scikit Learn                                                    ##
#####################################################################

In [ ]:
dt_model = DecisionTreeRegressor()
dt_model.fit(x_train_values, y_train_values)
dt_model_predictions = dt_model.predict(x_test_values)
generate_submission_file(dt_model_predictions, test_data["Id"], "../results/" + user + "_Decision_Tree.csv")

In [ ]:
param_list = {"max_depth": np.linspace(10, len(x_columns), 100, dtype=np.int64)}
dt_gridsearch = GridSearchCV(DecisionTreeRegressor(), param_list)
dt_gridsearch.fit(x_train_values, y_train_values)
dt_best_model_predictions = dt_gridsearch.best_estimator_.predict(x_test_values)
generate_submission_file(dt_best_model_predictions, test_data["Id"], "../results/" + user + "_Decision_Tree_GridSearchCV.csv")

## Ensemble of Trees ##

Ensembles are methods that combine multiple machine learning models to create more powerful models. There are many models in the machine learning literature that belong to this category, but there are two ensemble models that have proven to be effective on a wide range of datasets for classification and regression, both of which use decision trees as their building blocks: random forests and gradient boosted decision trees.

### Random Forests ###

As we just observed, a main drawback of decision trees is that they tend to overfit the training data. Random forests are one way to address this problem. A random forest is essentially a collection of decision trees, where each tree is slightly different from the others. The idea behind random forests is that each tree might do a relatively
good job of predicting, but will likely overfit on part of the data. If we build many trees, all of which work well and overfit in different ways, we can reduce the amount of overfitting by averaging their results. This reduction in overfitting, while retaining the predictive power of the trees, can be shown using rigorous mathematics.

To implement this strategy, we need to build many decision trees. Each tree should do an acceptable job of predicting the target, and should also be different from the other trees. Random forests get their name from injecting randomness into the tree building to ensure each tree is different. There are two ways in which the trees in a random
forest are randomized: by selecting the data points used to build a tree and by selecting the features in each split test. Let’s go into this process in more detail.

**Building random forests:** To build a random forest model, you need to decide on the number of trees to build (the `n_estimators` parameter of `RandomForestRegressor` or `RandomForestClassifier` ). Let’s say we want to build 10 trees. These trees will be built completely independently from each other, and the algorithm will make different random choices for each tree to make sure the trees are distinct. To build a tree, we first take what is called a bootstrap sample of our data. That is, from our `n_samples` data points, we repeatedly draw an example randomly with replacement (meaning the
same sample can be picked multiple times), `n_samples` times. This will create a dataset that is as big as the original dataset, but some data points will be missing from it (approximately one third), and some will be repeated. 

To illustrate, let’s say we want to create a bootstrap sample of the list `['a', 'b', 'c', 'd']` . A possible bootstrap sample would be `['b', 'd', 'd', 'c']` . Another possible sample would be ['d', 'a', 'd', 'a']. Next, a decision tree is built based on this newly created dataset. However, the algorithm we described for the decision tree is slightly modified. Instead of looking for the best test for each node, in each node the algorithm randomly selects a subset of the features, and it looks for the best possible test involving one of these features. The number of features that are selected is controlled by the `max_features` parameter. This selection of a subset of features is repeated separately in each node, so that each node in a tree can make a decision using a different subset of the features.

The bootstrap sampling leads to each decision tree in the random forest being built on a slightly different dataset. Because of the selection of features in each node, each split in each tree operates on a different subset of features. Together, these two mechanisms ensure that all the trees in the random forest are different. A critical parameter in this process is `max_features` . If we set `max_features` to `n_features`, that means that each split can look at all features in the dataset, and no randomness will be injected in the feature selection (the randomness due to the bootstrapping remains, though). If we set `max_features` to 1 , that means that the splits have no choice at all on which feature to test, and can only search over different thresholds for the feature that was selected randomly. Therefore, a high `max_features` means that the trees in the random forest will be quite similar, and they will be able to fit the data easily, using the most distinctive features. A low `max_features`
means that the trees in the random forest will be quite different, and that each tree might need to be very deep in order to fit the data well. 

To make a prediction using the random forest, the algorithm first makes a prediction for every tree in the forest. For regression, we can average these results to get our final prediction. For classification, a “soft voting” strategy is used. This means each algorithm makes a “soft” prediction, providing a probability for each possible output label. The probabilities predicted by all the trees are averaged, and the class with the highest probability is predicted.

In [ ]:
#####################################################################
# Random Forests                                                    #
#####################################################################

In [ ]:
#####################################################################
## Scikit Learn                                                    ##
#####################################################################

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(x_train_values, y_train_values)
rf_model_predictions = rf_model.predict(x_test_values)
generate_submission_file(rf_model_predictions, test_data["Id"], "../results/" + user + "_Random_Forests.csv")

In [ ]:
param_list = {"n_estimators": np.linspace(100, 500, 5, dtype=np.int64)}
rf_gridsearch = GridSearchCV(RandomForestRegressor(), param_list)
rf_gridsearch.fit(x_train_values, y_train_values)
rf_best_model_predictions = rf_gridsearch.best_estimator_.predict(x_test_values)
generate_submission_file(rf_best_model_predictions, test_data["Id"], "../results/" + user + "_Random_Forests_GridSearchCV.csv")

**Strengths, weaknesses, and parameters:**. Random forests for regression and classification are currently among the most widely used machine learning methods. They are very powerful, often work well without heavy tuning of the parameters, and don’t require scaling of the data.

Essentially, random forests share all of the benefits of decision trees, while making up for some of their deficiencies. One reason to still use decision trees is if you need a compact representation of the decision-making process. It is basically impossible to interpret tens or hundreds of trees in detail, and trees in random forests tend to be deeper than decision trees (because of the use of feature subsets). Therefore, if you need to summarize the prediction making in a visual way to nonexperts, a single decision tree might be a better choice. While building random forests on large datasets might be somewhat time consuming, it can be parallelized across multiple CPU

### Gradient Boosted Regression Trees (Gradient Boosting Machines) ###

Gradient boosted regression tree is another ensemble method that combines multiple decision trees to create a more powerful model. Despite the “regression” in the name, these models can be used for regression and classification. In contrast to the random forest approach, gradient boosting works by building trees in a serial manner, where each tree tries to correct the mistakes of the previous one. By default, there is no randomization in gradient boosted regression trees; instead, strong pre-pruning is used. Gradient boosted trees often use very shallow trees, of depth one to five, which makes the model smaller in terms of memory and makes predictions faster.

The main idea behind gradient boosting is to combine many simple models (in this context known as weak learners), like shallow trees. Each tree can only provide good predictions on part of the data, and so more and more trees are added to iteratively improve performance.
Gradient boosted trees are frequently the winning entries in machine learning competitions, and are widely used in industry. They are generally a bit more sensitive to parameter settings than random forests, but can provide better accuracy if the parameters are set correctly.

Apart from the pre-pruning and the number of trees in the ensemble, another important parameter of gradient boosting is the learning_rate , which controls how strongly each tree tries to correct the mistakes of the previous trees. A higher learning rate means each tree can make stronger corrections, allowing for more complex models. Adding more trees to the ensemble, which can be accomplished by increasing
`n_estimators` , also increases the model complexity, as the model has more chances to correct mistakes on the training set.

In [ ]:
#####################################################################
## Scikit Learn                                                    ##
#####################################################################

In [ ]:
gbm_model = GradientBoostingRegressor(random_state=0)
gbm_model.fit(x_train_values, y_train_values)
print("Accuracy on training set: {:.3f}".format(gbm_model.score(x_train_values, y_train_values)))
gbm_model_predictions = gbm_model.predict(x_test_values)
generate_submission_file(gbm_model_predictions, test_data["Id"], "../results/" + user + "_Gradient_Boosted_Machines.csv")

To reduce overfitting, we could either apply stronger pre-pruning by limiting the maximum depth or lower the learning rate.

In [ ]:
param_grid = {"n_estimators": np.linspace(10, 150, 15, dtype=np.int64)}
gbm_gridsearch = GridSearchCV(GradientBoostingRegressor(), param_grid)
gbm_gridsearch.fit(x_train_values, y_train_values)
gbm_best_model_predictions = gbm_gridsearch.best_estimator_.predict(x_test_values)
generate_submission_file(gbm_best_model_predictions, test_data["Id"], "../results/" + user + "_Gradient_Boosted_Machines_GridSearchCV.csv")

In [ ]:
#####################################################################
## XGBoost                                                         ##
#####################################################################

In [ ]:
#####################################################################
### Weak Learner is a Tree                                        ###
#####################################################################

As both gradient boosting and random forests perform well on similar kinds of data, a common approach is to first try random forests, which work quite robustly. If random forests work well but prediction time is at a premium, or it is important to squeeze out the last percentage of accuracy from the machine learning model, moving to gradient boosting often helps.

If you want to apply gradient boosting to a large-scale problem, it might be worth looking into the xgboost package and its Python interface, which is faster (and sometimes easier to tune) than the scikit-learn implementation of gradient boosting on many datasets.

In [ ]:
xgb_model = xgb.XGBRegressor()
xgb_model.fit(x_train_values, y_train_values)
xgb_model_predictions = xgb_model.predict(x_test_values)
generate_submission_file(xgb_model_predictions, test_data["Id"], "../results/" + user + "_XGBoost_Basic.csv")

In [ ]:
param_grid = {"max_depth": [2,4,6],
              "n_estimators": np.linspace(100, 500, 5, dtype=np.int64)}
xgb_grid_search = GridSearchCV(xgb.XGBRegressor(objective="reg:linear"), param_grid)
xgb_grid_search.fit(x_train_values, y_train_values)
xgb_model_predictions = xgb_grid_search.predict(x_test_values)
generate_submission_file(xgb_model_predictions, test_data["Id"], "../results/" + user + "_XGBoost_GridSearchCV.csv")

**Strengths, weaknesses, and parameters:** Gradient boosted decision trees are among the most powerful and widely used models for supervised learning. Their main drawback is that they require careful tuning of the parameters and may take a long time to train. Similarly to other tree-based models, the algorithm works well without scaling and on a mixture of binary and continuous features. As with other tree-based models, it also often does not work well on high-dimensional sparse data.

The main parameters of gradient boosted tree models are the number of trees, `n_estimators` , and the `learning_rate` , which controls the degree to which each tree is allowed to correct the mistakes of the previous trees. These two parameters are highly interconnected, as a lower learning_rate means that more trees are needed to build a model of similar complexity. In contrast to random forests, where a higher `n_estimators` value is always better, increasing n_estimators in gradient boosting leads to a more complex model, which may lead to overfitting. A common practice is to fit `n_estimators` depending on the time and memory budget, and then search over different `learning_rates`.
    
Another important parameter is max_depth (or alternatively `max_leaf_nodes` ), to reduce the complexity of each tree. Usually `max_depth` is set very low for gradient boosted models, often not deeper than five splits.